In [1]:
!pip install google-search-results

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms.huggingface_hub import HuggingFaceHub
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

/home/labicquette/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.agents import AgentExecutor

In [6]:
import os

#os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["SERPAPI_API_KEY"] = "c9ab8a0b29aa07ab3581e6d3258edadfa43c6c86d631d754527eaa54595b6d30"

In [7]:
from transformers import AutoTokenizer

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

In [8]:
import torch
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.43s/it]


In [9]:
"""llm = HuggingFaceHub.(
    repo_id=repo_id, model_kwargs={"max_length":200, 
                                   "do_sample":True,
                                    "top_k":10,
                                    "num_return_sequences":1,
                                    "eos_token_id":tokenizer.eos_token_id}
)
llm = HuggingFacePipeline.from_model_id(
    model_id="falcon-7b-instruct",task="text-generation", device_map="auto", model_kwargs={"max_length":200, 
                                   "do_sample":True,
                                    "top_k":10,
                                    "num_return_sequences":1,
                                    "eos_token_id":tokenizer.eos_token_id}
)"""
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

In [10]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [13]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handle_parsing_errors=True, verbose=True
)

In [14]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
